In [ ]:
import pandas as pd
import numpy as np
import scipy, scipy.stats
#import autograd
import matplotlib.pyplot as plt
import random
from collections import defaultdict

In [ ]:
df = pd.read_excel('fall19.xlsx')
df.Name = df.Name.map(lambda x: x.upper() if type(x) == str else _)
df_orig = df 


In [ ]:
dft=df[(df.Year ==2019) & (df.Semester == 'Fall') & (df.Dept == 'ROB')& (df.Level == 'Graduate')]

In [ ]:
#df[(df.Dept == 'ROB')& (df.Level == 'Graduate') & (df.Name == 'GALEOTTI, JOHN')]

In [ ]:
dft.groupby('Course ID').mean()['Overall course rate'].sort_values()

In [ ]:
course_code_to_dept = defaultdict(lambda : defaultdict(int))
for row in df.itertuples():
    course_code_to_dept[row[5][:2]][row[4]] += 1

In [ ]:
course_code_to_dept = {k: sorted([(c,x) for x,c in v.items()])[-1][1] for k,v in course_code_to_dept.items()}

In [ ]:
if False:
    DF_EXPAND = pd.read_excel('copied_data.xlsx')

    ID_lookup = {}
    for row in df.itertuples():
        ID_lookup[row[7]] = row[8]
    for row in DF_EXPAND.itertuples():
        if row[7] not in ID_lookup:
            ID_lookup[row[7]] = row[7].lower().replace(' ','').replace(',','')
            #print(ID_lookup[row[7]])
    DF_EXPAND['Instructor ID'] = DF_EXPAND['Name'].apply(lambda x: ID_lookup[x])
    df = pd.concat([df_orig,DF_EXPAND],sort=False,ignore_index=True)
df['Course ID'] = df['Course ID'].astype(str)

In [ ]:
df.loc[df['Course ID']== str(16697),'Course ID'] = str(16698)
df.loc[df['Course ID']== str(16730),'Course ID'] = str(16698)
df = df[df.Name.map(lambda x: type(x) == str)]

In [ ]:
df['TIME'] = (df['Year']-df['Year'].min())*2 + np.array(df['Semester'] == 'Fall').astype(np.int)
df['TIME'] = (df['TIME'] +1)
df['TIME'] = ((df['TIME'])/df['TIME'].max())
df.TIME = df.TIME**2
plt.style.use('ggplot')
plt.style.use('seaborn-white')
plt.plot(df.groupby('Year').mean()['TIME'],lw=4)
plt.ylabel('weight',size=20)
plt.xlabel('year',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.title('time weights',size=24)
plt.tight_layout()
plt.savefig('time-bias.pdf')


In [ ]:
subjects = list(df['Course ID'].unique())
reviewers = list(df['Name'].unique())
rm = df['Overall course rate'].mean()
rs = df['Overall course rate'].std()
rm,rs

In [ ]:

BANNED_NAMES = ['INDEPENDENT STUDY','INDEPENDENT STUDY','SEM.','SEMN','SEMNR','SP TPCS','SPEC TOPICS','READING','SPECIAL TOPICS','PRACTICUM','INTERNSHIP','SEMINAR','PROJECT','CAPSTONE']
#BANNED_NAMES = ['INDEPENDENT STUDY','INDEPENDENT STUDY','SEM.','SEMN','SEMNR','READING','PRACTICUM','INTERNSHIP','SEMINAR','PROJECT','CAPSTONE']

valid_names = df['Course Name'].apply(lambda x: sum([_ in x for _ in BANNED_NAMES]) == 0)
#df= df.loc[valid_names]

In [ ]:
subjectsD = {k:i for i,k in enumerate(subjects)}
reviewersD = {k:i for i,k in enumerate(reviewers)}
df.columns[4],df.columns[6],df.columns[23]

In [ ]:


n_sub = len(subjects)
n_rev = len(reviewers)
dataset = np.array(df)
total_reviews = np.zeros(n_sub)
num_reviews = np.zeros(n_sub)



for i in range(dataset.shape[0]):
    #print(dataset[i,4],dataset[i,6],dataset[i,23])
    dataset[i,0] = subjectsD[dataset[i,4]]
    dataset[i,1] = reviewersD[dataset[i,6]]
    #dataset[i,2] = (dataset[i,2]-rm)/rs
    
    # normalization step
    total_reviews[dataset[i,0]] += dataset[i,23]
    num_reviews[dataset[i,0]] += 1

average_reviews = total_reviews/num_reviews
np.random.shuffle(dataset)

In [ ]:
average_reviews

In [ ]:
reviewer_biases = np.random.randn(n_rev)
subject_scores = np.random.random(n_sub)

x0 = np.hstack([reviewer_biases,subject_scores])

In [ ]:
np.unique(dataset[:,0]).shape,n_sub,n_rev,np.unique(dataset[:,1]).shape,x0.shape

In [ ]:
max_w = np.sqrt(df.iloc[:,10].max())

In [ ]:
#df.columns[10]


In [ ]:
csize = df.iloc[:,10].unique()
csize = np.array(sorted(csize))

plt.style.use('seaborn-white')
plt.plot(csize,np.sqrt(csize)/max_w,lw=4)
plt.ylabel('weight',size=20)
plt.xlabel('number of respondents',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.title('size weights',size=24)
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.tight_layout()
plt.savefig('size-bias.pdf')


In [ ]:
#dept_means = df.loc[valid_names].groupby(['Dept','Level']).mean()

In [ ]:
#list(df.columns).index('Level')

In [ ]:
list(df.columns).index('Num Respondents'),list(df.columns).index('Dept')

In [ ]:
dept_bias = defaultdict(float)

dept_bias_cnt = defaultdict(float)
for row in df.loc[valid_names].itertuples():
    #print(row)
    key = row[4],row[9]
    val,num = row[24],row[12]
    #print(row[4],row[9],row[24],row[12])
    if np.isfinite(val):
        dept_bias[key] += val*num
        dept_bias_cnt[key] += num

for k in dept_bias:
    dept_bias[k]/=dept_bias_cnt[k]
#dept_bias

In [ ]:
course_bias = np.zeros(n_sub)
for row in df.itertuples():
    course_bias[subjectsD[row[5]]] = dept_bias[(row[4],row[9])]

In [ ]:
dept_names = sorted([_ for _ in dept_bias])
n_depts = len(dept_names)
n_dept_idx = {k:i for i,k in enumerate(dept_names)}

In [ ]:
Xd = {}
y = np.zeros(dataset.shape[0])
bias = np.zeros(dataset.shape[0])
for i,d in enumerate(dataset):
    if np.isfinite(d[23]):
        valid_bias = sum([_ in d[7] for _ in BANNED_NAMES]) ==0
        valid_bias = max(0.01,float(valid_bias))
        
        is_grad = int(d[8] == 'Graduate')*.5 + .5
        W =  valid_bias*is_grad*d[24]*np.sqrt(d[11])/max_w#np.sqrt(d[11]))
        Xd[(i,d[0])] = W
        Xd[(i,n_sub+d[1])] = W
        
        bias_idx = n_dept_idx[(d[3],d[8])]#['Overall course rate']
        Xd[(i,n_sub+n_rev+bias_idx)] = W
        y[i] = (d[23])*W


In [ ]:
#subjects
n_rev,n_sub,n_depts

In [ ]:

import scipy.sparse
import matplotlib.pyplot as plt

X = scipy.sparse.dok_matrix((dataset.shape[0],n_rev+n_sub+n_depts))
X._update(Xd)
X = scipy.sparse.csr_matrix(X)

In [ ]:
X.shape,y.max(),

In [ ]:
XT = X.T
Xt = XT @ X
Xy = XT @ y.reshape((-1,1))

In [ ]:
regV = np.ones(n_rev+n_sub+n_depts)*2.5e-2
regV[n_sub:] = 5e-2
regV[-n_depts:] = 0
x = scipy.linalg.solve(Xt + scipy.diag(regV), Xy, assume_a='pos')
diff = X.dot(x)[:,0] - y
abs(diff).mean(),np.linalg.norm(diff)

In [ ]:
mse = np.mean(diff**2)
mse

In [ ]:
erm = scipy.linalg.inv(Xt + scipy.diag(regV))

In [ ]:
sec = np.sqrt(np.diag(mse*erm))

In [ ]:
cv = scipy.stats.t.ppf(0.95,df=len(diff))
CI = cv*sec

In [ ]:
_ = plt.hist(diff,100)
plt.xlim(-.3,.3)
plt.grid(True)
plt.ylabel('number of examples',size=20)
plt.xlabel('prediction error',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.title('Distribution of errors',size=24)

plt.tight_layout()
plt.savefig('prediction-error.pdf')


In [ ]:
if False:
    xv = np.random.uniform(-6,-1,size=6)
    xv = np.sort(xv)
    Vs = []
    for lr in xv:
        regV = np.ones(n_rev+n_sub)*1e-3
        regV[n_sub:] = (10**(lr))
        x = scipy.linalg.solve(Xt + scipy.diag(regV), Xy, assume_a='pos')
        diff = X.dot(x)[:,0] - y
        Vs.append((lr,abs(diff).mean(),np.linalg.norm(diff)))

In [ ]:
if False:
    Vs = np.array(Vs)
    plt.plot(Vs[:,0],Vs[:,1])
    plt.figure()
    plt.plot(Vs[:,0],Vs[:,2])

In [ ]:
reg_dept_bias= x[-n_depts:,0]
reviewer_biases = x[n_sub:-n_depts,0]
course_offsets = x[:n_sub,0]

CI_reg_dept_bias= CI[-n_depts:]
CI_reviewer_biases = CI[n_sub:-n_depts]
CI_course_offsets = CI[:n_sub]


In [ ]:
for row in df.itertuples():
    course_bias[subjectsD[row[5]]] = reg_dept_bias[n_dept_idx[(row[4],row[9])]]

In [ ]:
subject_scores = course_offsets + course_bias

In [ ]:
for n in dept_names:
    print(n,reg_dept_bias[n_dept_idx[n]],dept_bias[n])

In [ ]:
_ = plt.hist(reviewer_biases,50)
plt.ylabel('number of examples',size=20)
plt.xlabel('instructor bias',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.xlim(-1,1)
plt.title('instructor offsets',size=24)
plt.tight_layout()
plt.savefig('pred-inst.pdf')

plt.figure()
_ = plt.hist(x[:n_sub,0],50)
plt.ylabel('number of examples',size=20)
plt.xlabel('course bias from department average',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.xlim(-1,1)
plt.title('course offsets',size=24)

plt.tight_layout()
plt.savefig('pred-offset.pdf')

plt.figure()
_ = plt.hist(subject_scores,50)
plt.ylabel('number of examples',size=20)
plt.xlabel('predicted "core" course scores',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.title('subject scores',size=24)
plt.xlim(2.5,5.5)

plt.tight_layout()
plt.savefig('pred-scores.pdf')

plt.figure()
_ = plt.hist(reg_dept_bias,50)
plt.ylabel('number of examples',size=20)
plt.xlabel('dept average',size=20)
plt.gca().tick_params(axis='both', which='major', labelsize=16)
plt.gca().tick_params(axis='both', which='minor', labelsize=16)
plt.grid(True)
plt.title('dept average scores',size=24)
plt.xlim(3,5)

plt.tight_layout()
plt.savefig('dept_bias.pdf')

In [ ]:
base_set = df[(df.Level == 'Graduate') & ((df.Dept == 'ROB') | (df.Dept == 'zzz'))]
valid_Iids = set(base_set['Name'].unique())
valid_Cids = set(base_set['Course ID'].unique())
df.Dept.unique()

In [ ]:
#for n in df[df.Dept == 'ROB']['Course Name']:
#    print(n)

In [ ]:
from collections import defaultdict
class_lookup = defaultdict(list)
name_lookup = defaultdict(list)

for row in df.itertuples():
    #print(row[7],'\t',row[5],'\t',row[4],'\t',row[8])
    #name_lookup[row[7]].append(row[6])
    class_lookup[row[5]].append(row[8])

In [ ]:
dept_bias[('MEG','Graduate')],dept_bias[('MLG','Graduate')],dept_bias[('ROB','Graduate')],dept_bias[('STA','Graduate')],dept_bias[('CS','Graduate')],dept_bias[('HCI','Graduate')]

In [ ]:
#reviewers[i]
#name_lookup
len(reviewer_biases),len(reviewers)

In [ ]:

print('{:40s}\t\t{}\t{}'.format('name','bias','95%'))

for i in np.argsort(reviewer_biases):
    if reviewers[i] not in valid_Iids:
        continue
    print('{:40s}\t\t{:.2f}\t{:.2f}'.format(reviewers[i],reviewer_biases[i],CI_reviewer_biases[i]))

In [ ]:
subjects[i]

In [ ]:
print('{:40s}\t\t{}\t{}'.format('class','score','95%'))
scores = []
for i in np.argsort(subject_scores):
    if subjects[i] not in valid_Cids:
        continue
    if subject_scores[i] == 0:
        continue
    scores.append(subject_scores[i])
    print('{:40s}\t\t{:.2f}\t{:.2f}'.format(class_lookup[subjects[i]][0],subject_scores[i],CI_course_offsets[i]))
scores = np.array(scores)
scores.mean(),scores.std()

In [ ]:
average_reviews

In [ ]:
import matplotlib.pyplot as plt
if False:
    plt.figure(figsize=(8,8))
    #plt.scatter(average_reviews,subject_scores+(average_reviews.mean()-subject_scores.mean()))
    plt.scatter(average_reviews,subject_scores)
    plt.xlabel('average review')
    plt.ylabel('true review')
    maxv = max(abs(average_reviews).max(),abs(subject_scores).max())
    #plt.xlim(-maxv,maxv)
    #plt.ylim(-maxv,maxv)
    plt.grid(True)
    plt.show()

In [ ]:
short_lookup = defaultdict(list)
for name in set(reviewers):
    if type(name) == str:
        last = name.split(', ')[0]
        short_lookup[last].append(name)

In [ ]:
targets = pd.read_excel('courses_of_interest.xlsx') #courses_of_interest #spring_draft

In [ ]:
targets

In [ ]:
import scipy.stats
short_lookup['OTOOLE']


In [ ]:
results = []
total_scores_vec = df.loc[valid_names  & (df['Num Respondents'] >= 15)  ]
for row in targets.itertuples():
    cv = subject_scores[subjectsD[str(row[1])]] if str(row[1]) in subjectsD else 0
    row_dept = str(row[1])
    if cv == 0:
        cv = {'36': dept_bias[('STA','Graduate')],'10': dept_bias[('MLG','Graduate')], '16': dept_bias[('ROB','Graduate')]}[row_dept[:2]]
    #print(row[-1])
    nv = []
    for name in row[-1].split(', '):
        lookup_name = name.upper().replace("'",'')
        list_of_names = short_lookup[lookup_name]
    
        if ';' in lookup_name:
            new_name = lookup_name.replace(';',', ')
            if new_name in reviewersD:
                list_of_names = [new_name]
        
        if len(list_of_names) == 0:
            print('didnt find ' + lookup_name)
            print(row)
        if len(list_of_names) > 1:
            print('ambig ', list_of_names)

        nv.append(list_of_names)

    nvL = [reviewer_biases[reviewersD[_[0]]] if len(_) > 0 else 0 for _ in nv ]
    nv2 = nv
    nv = np.mean(nvL) if len(nvL) > 0 else 0
    TR = cv + nv if len(nvL) >0 else 0
    if TR <=0.1:
        print(row)
        continue
    new_name = ','.join([ _.split(';')[0] if ';' in _ else _ for _ in row[-1].split(',')])
    #print(new_name)
    d = {'num':row[1],'name':row[2],'Instructor':new_name,'Instructor Rating':nv,'Course Rating':cv,'Total Rating':TR}
    d['course %'] = scipy.stats.percentileofscore(subject_scores,cv)
    d['instru %'] = scipy.stats.percentileofscore(reviewer_biases,nv)
    d['total %'] = scipy.stats.percentileofscore(total_scores_vec['Overall course rate'].fillna(0),TR)
    
    CIss = CI_course_offsets[subjectsD[str(row[1])]] if str(row[1]) in subjectsD else 0.5
    CIL = [CI_reviewer_biases[reviewersD[_[0]]] if len(_) > 0 else 0 for _ in nv2 ]
    CIcs = np.mean(CIL) if len(CIL) > 0 else 0.5
    bd=  CIcs+ CIss
    d['lb'] = TR - bd
    d['ub'] = TR + bd
    results.append(d)
#RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating','course %','instru %','total %']]
RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating','lb','ub']]

#RES = pd.DataFrame(results)[['num','name','Instructor','course %','instru %','total %']]
RES = RES.sort_values('Total Rating',0,False)
RES['num'] = RES['num'].astype(str)
RES['num'] = RES['num'].apply(lambda x: x[:2] + '-' + x[2:])
#RES.columns = ['num', 'name', 'Instructor', 'Course Estimate', 'Instructor Estimate', 'Total Estimate']
RES.round(2).style.background_gradient(cmap='RdYlGn')
#RES.round(0).style.background_gradient(cmap='RdYlGn',low=0)

In [ ]:
RES.to_excel('output_p2.xlsx',index=False)

In [ ]:
df.loc[df.Name.str.contains('WHITTAKER'.upper()).fillna(False)] 


In [ ]:
df[df['Course ID'] == str(16881)]

In [ ]:
df[df['Course Name'] == 'ROBOTICS BIZ']

In [ ]:
 targets = pd.read_excel('mech-e-spring.xlsx') #courses_of_interest #spring_draft stats_clsases spring2020

In [ ]:
results = []
for row in targets.itertuples():
    #print(row)
    cv = subject_scores[subjectsD[str(row[1])]] if str(row[1]) in subjectsD else 0
    row_dept = str(row[1])
    if cv == 0:
        cv = {'24':reg_dept_bias[n_dept_idx[('MEG','Graduate')]],
              '36':reg_dept_bias[n_dept_idx[('STA','Graduate')]],
              '10':reg_dept_bias[n_dept_idx[('MLG','Graduate')]], 
              '16': reg_dept_bias[n_dept_idx[('ROB','Graduate')]]}[row_dept[:2]]
    #print(row[-1])
    nv = []
    for fname in row[-1].split(';'):
        fname = fname.replace("'",'')
        sname = fname.upper().split(',')
        first_name = sname[-1]
        last_name = sname[0]
        #print(first_name,last_name)
        
        list_of_names = short_lookup[last_name]  
        #print(fname,list_of_names)
        if fname.upper() in list_of_names:
            #print(fname)
            list_of_names = [fname.upper()]
        #print(fname,list_of_names,len(list_of_names))

        if len(list_of_names) == 0:
            print('didnt find ' + fname)
        if len(list_of_names) > 1:
            print('ambig ',fname, list_of_names)
            list_of_names = []
        nv.append(list_of_names)
    #print(row[-1],nv)
    nvL = [reviewer_biases[reviewersD[_[0]]] if len(_) > 0 else 0 for _ in nv ]
    nv = np.mean(nvL) if len(nvL) > 0 else 0
    TR = cv + nv if len(nvL) >0 else 0
    if TR <=0.1:
        print(row)
        continue
    new_name = ', '.join([ _.split(',')[0] if ',' in _ else _ for _ in row[-1].split(';')])
    #print(new_name)
    #print(new_name)
    d = {'num':row[1],'name':row[2],'Instructor':new_name,'Instructor Rating':nv,'Course Rating':cv,'Total Rating':TR}
    d['course %'] = scipy.stats.percentileofscore(subject_scores,cv)
    d['instru %'] = scipy.stats.percentileofscore(reviewer_biases,nv)
    d['total %'] = scipy.stats.percentileofscore(total_scores_vec['Overall course rate'].fillna(0),TR)

    results.append(d)
#RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating','course %','instru %','total %']]
RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating']]

#RES = pd.DataFrame(results)[['num','name','Instructor','course %','instru %','total %']]
RES = RES.sort_values('Total Rating',0,False)
RES['num'] = RES['num'].astype(str)
RES['num'] = RES['num'].apply(lambda x: x[:2] + '-' + x[2:])
#RES.columns = ['num', 'name', 'Instructor', 'Course Estimate', 'Instructor Estimate', 'Total Estimate']
RES.round(2).style.background_gradient(cmap='RdYlGn')
#RES.round(0).style.background_gradient(cmap='RdYlGn',low=0)

In [ ]:
short_lookup['Liu'.upper()]

In [ ]:
RES.round(2).style.background_gradient(cmap='RdYlGn').to_excel('output_MESPRING.xlsx',index=False)

In [ ]:
html_data = pd.read_html('fall_2019_grad.html')

In [ ]:
#html_data[-3].Instructor[10].replace('  ',';')

In [ ]:
for i in range(len(html_data)):
    html_data[i].dropna(subset=['Course'],inplace=True)
    html_data[i].loc[:,'Course'] = html_data[i].Course.map(lambda x: str(int(x)) if np.isfinite(x) else str(x))
    html_data[i].loc[:,'Instructor'] = html_data[i].Instructor.map(lambda x: x.replace('  ',';'))


In [ ]:
for i in range(len(html_data)):
    html_data[i].loc[:,'Course'] = html_data[i].Course.map(lambda x: '0' + x if len(x) ==4 else x)

In [ ]:
all_results = {}
for d in html_data:
    results = []
    DEPT_NUM = d.loc[0].Course[:2]
    print('\n\nNOW DOING THE {} DEPARTMENT\n'.format(DEPT_NUM))
    for row in d.itertuples():
        #print(row)
        cv = subject_scores[subjectsD[str(row[1])]] if str(row[1]) in subjectsD else 0
        row_dept = str(row[1])
        if cv == 0:
            if row_dept[:2] in course_code_to_dept:
                cv = reg_dept_bias[n_dept_idx[(course_code_to_dept[row_dept[:2]],'Graduate')]]
            else:
                #print(row)
                cv = reg_dept_bias.mean()
        #print(row[-1])
        nv = []
        for fname in row[-1].split(';'):
            fname = fname.replace("'",'')
            sname = fname.upper().split(',')
            first_name = sname[-1]
            last_name = sname[0]
            #print(first_name,last_name)

            list_of_names = short_lookup[last_name]  
            #print(fname,list_of_names)
            if fname.upper() in list_of_names:
                #print(fname)
                list_of_names = [fname.upper()]
            #print(fname,list_of_names,len(list_of_names))

            if len(list_of_names) == 0:
                print('didnt find ' + fname)
            if len(list_of_names) > 1:
                print('ambig ',fname, list_of_names)
                list_of_names = []
            nv.append(list_of_names)
        #print(row[-1],nv)
        nvL = [reviewer_biases[reviewersD[_[0]]] if len(_) > 0 else 0 for _ in nv ]
        nv = np.mean(nvL) if len(nvL) > 0 else 0
        TR = cv + nv if len(nvL) >0 else 0
        if TR <=0.1:
            print(row)
            continue
        new_name = ', '.join([ _.split(',')[0] if ',' in _ else _ for _ in row[-1].split(';')])
        #print(new_name)
        #print(new_name)
        d = {'num':row[1],'name':row[2],'Instructor':new_name,'Instructor Rating':nv,'Course Rating':cv,'Total Rating':TR}
        d['course %'] = scipy.stats.percentileofscore(subject_scores,cv)
        d['instru %'] = scipy.stats.percentileofscore(reviewer_biases,nv)
        d['total %'] = scipy.stats.percentileofscore(total_scores_vec['Overall course rate'].fillna(0),TR)

        results.append(d)
    #RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating','course %','instru %','total %']]
    RES = pd.DataFrame(results)[['num','name','Instructor','Course Rating','Instructor Rating','Total Rating']]

    #RES = pd.DataFrame(results)[['num','name','Instructor','course %','instru %','total %']]
    RES = RES.sort_values('Total Rating',0,False)
    RES['num'] = RES['num'].astype(str)
    RES['num'] = RES['num'].apply(lambda x: x[:2] + '-' + x[2:])
    #RES.columns = ['num', 'name', 'Instructor', 'Course Estimate', 'Instructor Estimate', 'Total Estimate']
    all_results[DEPT_NUM] =RES
    #RES.round(0).style.background_gradient(cmap='RdYlGn',low=0)

In [ ]:
all_results['16'].round(2).style.background_gradient(cmap='RdYlGn').to_excel('pred_tmp.xlsx')

In [ ]:
all_results['16'].set_index('num')['Total Rating']

In [ ]:
dft['Course ID'] = dft['Course ID'].map(lambda x: str(x)[:2] + '-' + str(x)[2:] )

In [ ]:
debugr = all_results['16'].set_index('num')
debugr['actual'] = dft.groupby('Course ID').mean()['Overall course rate']
debugr['error'] = debugr['actual'] - debugr['Total Rating']

In [ ]:
#debugr['updated pred'] = all_results['16'].set_index('num')['Total Rating']

In [ ]:
debugr.dropna().round(2).style.background_gradient(cmap='RdYlGn')

In [ ]:
_ = plt.hist(debugr['error'],5)
plt.title('sigma: {:.2f}'.format(debugr['error'].std()))

In [ ]:
debugr.dropna().round(2).style.background_gradient(cmap='RdYlGn')#.to_excel('update.xlsx')